In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

def adversarial_attack(model, data, epsilon):
    data.requires_grad = True
    output = model(data)
    loss = nn.CrossEntropyLoss()(output, torch.argmax(data, dim=1))
    model.zero_grad()
    loss.backward()
    perturbed_data = data + epsilon * data.grad.sign()
    return perturbed_data.detach()

def train_with_adversarial_examples(train_loader, epochs=10, epsilon=0.1):
    model = SimpleNN()
    optimizer = optim.Adam(model.parameters())
    
    for epoch in range(epochs):
        for data, target in train_loader:
            optimizer.zero_grad()
            perturbed_data = adversarial_attack(model, data, epsilon)
            output = model(perturbed_data)
            loss = nn.CrossEntropyLoss()(output, target)
            loss.backward()
            optimizer.step()

# train_loader = your_data_loader  # Placeholder for actual data loader
# train_with_adversarial_examples(train_loader)